In [1]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

In [41]:
feim = pd.read_csv('../output/valid_single_feature_tochi.csv', header=None)
# feim = pd.read_csv('../output/valid_single_feature.csv', header=None)
feim.columns = ['datetime', 'DT-M', 'feature', 'score']

feim = feim[feim['datetime']>='20190916_00000']
print(feim['datetime'].max())

feim.set_index('feature', inplace=True)
cnt = feim.groupby('feature')['score'].count()
idx = cnt[cnt>=3].index
feim = feim.loc[idx]
may = feim[feim['DT-M']=='2018-5']
may.sort_values(by='score', ascending=False, inplace=True)
feim.loc[may.index]

20190916_16174


,datetime,DT-M,score
feature,,,
612__cnt_uid3_D11_mean_train,20190916_10263,2018-5,0.939072
612__cnt_uid3_D11_mean_train,20190916_10263,2018-4,0.947671
612__cnt_uid3_D11_mean_train,20190916_10263,2018-3,0.937936
612__uid3_C6_C14_D2_std_train,20190916_09591,2018-5,0.939028
612__uid3_C6_C14_D2_std_train,20190916_09591,2018-4,0.945548
612__uid3_C6_V187_V76_mean_train,20190916_15004,2018-5,0.938944
612__uid3_C6_V187_V76_mean_train,20190916_15004,2018-4,0.942961
612__cnt_uid3_C6_D10_mean_train,20190916_09043,2018-5,0.938902
612__cnt_uid3_C6_D10_mean_train,20190916_09043,2018-4,0.947717


In [43]:
tmp = feim[feim['DT-M']=='2018-3']
list_feature = tmp[tmp['score']>0.940].index
# list_feature = tmp[(0.937 < tmp['score']) & (tmp['score'] < 0.938)].index
# from_dir = 'product_feature'
# from_dir = 'check_trush'
from_dir = 'org_trush'
from_dir = 'create'
to_dir = 'org_use'
to_dir = 'valid_use'

for feature in list_feature:
    print(feature)
    move_feature([feature.replace('_train', '')], from_dir, to_dir)

612__cnt_uid3_C14_V187_V133_mean_train


array([5])

In [20]:
tmp[tmp['score']>0.937]

,datetime,DT-M,score
feature,,,
509__C11-V44__ratio__ProductCD-C_train,20190914_06201,2018-5,0.937060
512__V127-V318__ratio__ProductCD-W_train,20190914_13211,2018-5,0.938651
512__V127-V78__diff__ProductCD-C_train,20190914_16115,2018-5,0.937358
512__V130-V281__diff__ProductCD-H_train,20190914_06140,2018-5,0.937180
512__V53-V78__diff__ProductCD-W_train,20190914_13155,2018-5,0.937122
603__addr1-addr2_C8__ProductCD-H_std_train,20190914_16063,2018-5,0.937199
603__card1-card4_C6__ProductCD-C_mean_train,20190914_07480,2018-5,0.937604
603__card3-card6_C9__ProductCD-C_std_train,20190914_06595,2018-5,0.937283
603__card6-addr2_C4__ProductCD-C_std_train,20190914_12505,2018-5,0.937022
